# Init everything

In [1]:
import os
os.environ["ROS_DOMAIN_ID"]="0"

import asyncio
import pyjs
import rclpy

from rclpy.utilities import get_default_context
from rclpy.signals import SignalHandlerOptions
from rclpy.signals import install_signal_handlers
from rclpy.node import Node
from std_msgs.msg import String

class MinimalSubscriber(Node):
  def __init__(self, name, topics):
    super().__init__(name)
    self._subs = [self.create_subscription(
      String,
      topic,
      self.listener_callback,
      10) for topic in topics]
    self._subs  # prevent unused variable warning
    self._name = name

  def listener_callback(self, msg):
    print(self._name + " heard: \"" + msg.data + "\"")

# Global running flag used to stop async tasks
running = True

# Async spin/publish functions
async def spin_subscriber(sub):
  while running:
    rclpy.spin_once(sub, timeout_sec=0)
    await asyncio.sleep(0.01)

async def publish_messages(pub, text):
  count = 0
  while running:
    msg = String()
    msg.data = text + str(count)
    count = count + 1
    pub.publish(msg)
    await asyncio.sleep(1)

# Create ROS context
context = get_default_context()
context.init(args=None, domain_id=None)
context.ok()

# Create publishers and subscribers
node = rclpy.create_node('minimal_publisher', context=context, use_global_arguments=False, start_parameter_services=True)
pub_a = node.create_publisher(String, 'topic_a', 10)
pub_b = node.create_publisher(String, 'topic_b', 10)
sub_1 = MinimalSubscriber('minimal_subscriber_test', ['topic_a', 'topic_b'])
sub_2 = MinimalSubscriber('minimal_subscriber_test_a', ['topic_b'])

# Run publishers/subscribers

In [2]:
running = True
print("Running nodes for 5 seconds")
asyncio.create_task(publish_messages(pub_a, 'Hello From A '))
asyncio.create_task(publish_messages(pub_b, 'Hello From B '))
asyncio.create_task(spin_subscriber(sub_1))
asyncio.create_task(spin_subscriber(sub_2))

async def stopper():
    global running
    await asyncio.sleep(5)
    print("Stopping all tasks")
    running = False
    
asyncio.create_task(stopper())

Running nodes for 5 seconds


<Task pending name='Task-5' coro=<stopper() running at /tmp/xpython_42/1230654137.py:8>>

minimal_subscriber_test heard: "Hello From A 0"
minimal_subscriber_test heard: "Hello From B 0"
minimal_subscriber_test_a heard: "Hello From B 0"
minimal_subscriber_test_a heard: "Hello From B 1"
minimal_subscriber_test heard: "Hello From A 1"
minimal_subscriber_test heard: "Hello From B 1"
minimal_subscriber_test_a heard: "Hello From B 2"
minimal_subscriber_test heard: "Hello From A 2"
minimal_subscriber_test heard: "Hello From B 2"
minimal_subscriber_test heard: "Hello From A 3"
minimal_subscriber_test heard: "Hello From B 3"
minimal_subscriber_test_a heard: "Hello From B 3"
minimal_subscriber_test heard: "Hello From A 4"
minimal_subscriber_test heard: "Hello From B 4"
minimal_subscriber_test_a heard: "Hello From B 4"
Stopping all tasks
